In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from freezegun import freeze_time

from src import training_week_generation

# Sunday Night Training Week Message

In [6]:
@freeze_time("2024-08-11 20:00:00")
def trigger_training_week_generation():
    return training_week_generation.lambda_handler({}, {})

trigger_training_week_generation()

athlete_id='98390356' token still valid until 2024-08-13 04:04:40+00:00


/Users/jamievoynow/Desktop/code/strava/src/activities.py:113: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  .with_columns(


# Mid-Week Update

In [58]:
from src.types.training_week import TrainingWeek
from src.types.training_week_with_coaching import TrainingWeekWithCoaching
import polars as pl

def calc_weekly_total_mileage(training_week: TrainingWeek) -> float:
    """
    Calculates the total mileage for a given week

    :param training_week: instance of TrainingWeek
    :return: The total mileage
    """ 
    return sum(
        [
            getattr(training_week, attrib).distance 
            for attrib in training_week.__dict__
        ]
    )


def get_mid_week_delta(
    training_week_with_coaching: TrainingWeekWithCoaching,
    activities: pl.DataFrame
) -> float:
    """
    Assumes that activities is truncated to the current week, calculating the 
    difference between target mileage and actual mileage thus far this week

    :param training_week_with_coaching: instance TrainingWeekWithCoaching
    :param activities: DataFrame of activities thus far this week
    :return: The mileage delta as a float
    """
    actual_mileage = activities['distance_in_miles'].sum()
    weekly_target = calc_weekly_total_mileage(
        training_week_with_coaching.training_week
    )
    mileage_delta = weekly_target - actual_mileage

    return mileage_delta

In [53]:
@freeze_time("2024-08-08 20:00:00")
def mock_get_activities_df(strava_client, num_weeks=8):
    return get_activities_df(strava_client, num_weeks)

training_week_with_coaching = get_training_week_with_coaching(athlete_id)
activities = mock_get_activities_df(strava_client, num_weeks=1)

mid_week_detla = get_mid_week_delta(
    training_week_with_coaching=training_week_with_coaching,
    activities=activities
)